In [1]:
spark

In [24]:
from pyspark.sql.functions import col, expr, round, when

dir0 = 'file:///home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/feature_creation/' \
       + '02a.problematic_members_excluded/'

def read_and_select(file_flag):
    global dir0
    df = spark.read.format('csv').option('header','true').load(dir0 + file_flag) \
         .select('tid', 'msno', 'trans_date', 'pay_method', 'plan_days',
                 'list_price', 'actual_paid',
                 (col('list_price').cast('int') - col('actual_paid').cast('int')).alias('discount'),
                 round(col('actual_paid').cast('int')/col('plan_days'), 3).alias('paid_per_day'),
                 when(col('auto_renew')=='true', '1').otherwise('0').alias('auto_renew')
                )
    return df

df0 = read_and_select('msno_a-n')
df1 = read_and_select('msno_o-z')
df2 = read_and_select('msno_A-N')
df3 = read_and_select('msno_O-Z')
df4 = read_and_select('msno_non-letter')

df_all = df0.union(df1).union(df2).union(df3).union(df4)

outfile = 'file:///home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/feature_creation/' \
          + '02b.w_features_added'
df_all.write.format('csv').option('header','true').save(outfile)